## Concept Similarity with WordNet

In [ ]:
import nltk 
from nltk.corpus import wordnet as wn 
import numpy as np 
import os 
from scipy.stats import spearmanr, pearsonr

In [ ]:
nltk.download('wordnet', quiet=True) 
nltk.download('omw-1.4', quiet=True)
depth_max = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

### Funzioni usate per calcolare la similarità
Maximum Similarity

In [ ]:
def max_similarity(term1, term2, similarity_function): 
    synsets1 = wn.synsets(term1) 
    synsets2 = wn.synsets(term2)
     
    max_sim = 0 
     
    for syn1 in synsets1: 
        for syn2 in synsets2: 
            sim = similarity_function(syn1, syn2) 
            if sim > max_sim: 
                max_sim = sim 
    return max_sim

Calcolo della similarità di Wu-Palmer

In [ ]:
def wu_palmer_similarity(syn1, syn2): 
    lcs_candidates = syn1.lowest_common_hypernyms(syn2) 
    if not lcs_candidates: 
        return 0 
    lcs = lcs_candidates[0] 
    depth_lcs = lcs.max_depth() 
    depth_syn1 = syn1.max_depth() 
    depth_syn2 = syn2.max_depth() 
    return (2 * depth_lcs) / (depth_syn1 + depth_syn2)

Calcolo della Shortest path similarity

In [ ]:
def shortest_path_similarity(syn1, syn2):
    path_length = syn1.shortest_path_distance(syn2)
    if path_length is None:
        return 0
    return 2 * depth_max - path_length

Calcolo della similarità di Leacock&Chodorow

In [ ]:
def leacock_chodorow_similarity(syn1, syn2):
    if depth_max == 0: 
        return 0 
    path_length = syn1.shortest_path_distance(syn2) 
    if path_length is None or path_length == 0:
        return 0 
    return -np.log((path_length) / (2 * depth_max))

Indici di correlazione di Spearman e Pearson


In [ ]:
def calculate_correlations(list, similarity_function): 
    similarities = [] 
    human_ratings = [float(row[2]) / 10 for row in list] 
     
    for row in list: 
        sim = max_similarity(row[0], row[1], similarity_function) 
        similarities.append(sim) 
         
    spearman_corr = spearmanr(human_ratings, similarities) 
    pearson_corr = pearsonr(human_ratings, similarities) 
    #print(human_ratings) 
    #print(similarities) 
    return spearman_corr, pearson_corr

### Calcolo delle metriche di similarità + indici di correlazione
Lettura dei dati da 'WordSim353.csv'

In [ ]:
current_dir = os.getcwd()
file_name = 'WordSim353.csv'
 
file_path = os.path.join(current_dir, 'cc', file_name)

with open(file_name, 'r', encoding='utf-8') as train: 
   righe = train.readlines()[1:] 
 
combo_parole = [] 
 
for riga in righe: 
    riga = riga.strip().split(',') 
    combo_parole.append([riga[0], riga[1], riga[2]]) 
 
print(combo_parole)



Calcolo e stampa dei risultati

In [ ]:
spearman_wup, pearson_wup = calculate_correlations(combo_parole, wu_palmer_similarity) 
spearman_path, pearson_path = calculate_correlations(combo_parole, shortest_path_similarity) 
spearman_lch, pearson_lch = calculate_correlations(combo_parole, leacock_chodorow_similarity)

print(f"Wu & Palmer - Spearman: {spearman_wup}, Pearson: {pearson_wup}") 
print(f"Shortest Path - Spearman: {spearman_path}, Pearson: {pearson_path}") 
print(f"Leacock & Chodorow - Spearman: {spearman_lch}, Pearson: {pearson_lch}")

In [ ]:
print('[S1: {}] {}\n'.format(s1, s1.definition()))
print('[S1: {}] {}\n'.format(s2, s2.definition()))

print('wup_similarity({}, {}) = {}'.format(s1, s2, s1.wup_similarity(s2)))
print('lch_similarity({}, {}) = {}'.format(s1, s2, s1.lch_similarity(s2)))
print('path_similarity({}, {}) = {}'.format(s1, s2, s1.path_similarity(s2)))